In [1]:
import librosa
import matplotlib.pyplot as plt
import glob
import numpy as np
import soundfile
import os

In [2]:
values = ['neutral','calm','happy','sad','angry','fearful','disgust','surprised']
needed = ["happy", "sad", "angry", "calm", "fearful"]


In [3]:
x, y = [], []
for i in glob.glob("speech-emotion-recognition/Actor_*/*.wav"):
         name=os.path.basename(i)
         emotion=values[int(name.split("-")[2])-1]
         if emotion in needed:
                with soundfile.SoundFile(i) as sound_file:
                    X = sound_file.read(dtype="float32")
                    sample_rate = sound_file.samplerate
                    stft = np.abs(librosa.stft(X))
                    result = np.array([])
                    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
                    result = np.hstack((result, mfccs))
                    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
                    result = np.hstack((result, chroma))
                    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
                    result = np.hstack((result, mel))
                x.append(result)
                y.append(emotion)


In [4]:
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import OneHotEncoder

In [5]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [6]:
encoder=OneHotEncoder()
ye_train = encoder.fit_transform(list(map(lambda x: [x], y_train))).toarray()
ye_test = encoder.fit_transform(list(map(lambda x: [x], y_test))).toarray()


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [15]:
sequential=Sequential()
sequential.add(Dense(64, activation="relu", input_shape=np.array(X_train[0]).shape))
sequential.add(Dense(128, activation="relu"))
sequential.add(Dense(64, activation="sigmoid"))
sequential.add(Dense(128, activation="relu"))
sequential.add(Dense(len(needed), activation="softmax"))
sequential.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
sequential.fit(np.array(X_train),np.array(ye_train),epochs=200,batch_size=25)


Train on 672 samples
Epoch 1/200
672/672 [==============================] - 1s 838us/sample - loss: 1.6320 - accuracy: 0.2202
Epoch 2/200
672/672 [==============================] - 0s 216us/sample - loss: 1.5796 - accuracy: 0.2634
Epoch 3/200
672/672 [==============================] - 0s 241us/sample - loss: 1.5210 - accuracy: 0.3259
Epoch 4/200
672/672 [==============================] - 0s 239us/sample - loss: 1.4689 - accuracy: 0.3646
Epoch 5/200
672/672 [==============================] - 0s 207us/sample - loss: 1.4122 - accuracy: 0.3824
Epoch 6/200
672/672 [==============================] - 0s 214us/sample - loss: 1.3829 - accuracy: 0.4018
Epoch 7/200
672/672 [==============================] - 0s 242us/sample - loss: 1.3682 - accuracy: 0.4167
Epoch 8/200
672/672 [==============================] - 0s 244us/sample - loss: 1.3546 - accuracy: 0.3914
Epoch 9/200
672/672 [==============================] - 0s 246us/sample - loss: 1.3045 - accuracy: 0.4449
Epoch 10/200
672/672 [============

In [9]:
import pandas as pd

In [10]:
max(pd.DataFrame(sequential.history.history)["accuracy"])

0.9017857313156128

In [11]:
features_encoded = encoder.get_feature_names()

In [16]:
pred = sequential.predict(np.array(X_test))
pred_values=[]
for i in pred:
    pred_values.append(features_encoded[np.argmax(i)].replace("x0_",""))
len([1 for i, j in zip(y_test, pred_values) if i == j])/len(y_test)


0.6944444444444444

In [17]:
sequential.save("model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model/assets


2021-10-06 15:52:13.276791: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [14]:
features_encoded


array(['x0_angry', 'x0_calm', 'x0_fearful', 'x0_happy', 'x0_sad'],
      dtype=object)